In [1]:
import os
import cv2
import numpy as np
import pandas as pd

import image_processor
import model_builder

from tensorflow.python.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

In [2]:
image_size = (32, 32)
input_size = (*image_size, 1)

In [3]:
df_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
            'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
            'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
            'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
            'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

In [4]:
df_train = image_processor.process_image_data("data/Train", image_size, targets=df_names)

In [5]:
X = df_train.drop(columns=['target'])
y = df_train['target']

In [6]:
X_data = X.values.reshape(-1, *image_size, 1).astype('float32')
y_data = y.factorize()[0]

In [18]:
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.1, stratify=y_data, random_state=42)

In [21]:
model = model_builder.build_conv_pool_model(input_size, len(df_names))

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(X_train, y_train,
                    epochs=50,
                    batch_size=124,
                    callbacks=[early_stopping],
                    validation_data=(X_val, y_val),
                    verbose=1)

Epoch 1/50
748/748 [==============================] - 17s 23ms/step - loss: 1.0761 - accuracy: 0.7278 - val_loss: 0.5065 - val_accuracy: 0.8375
Epoch 2/50
748/748 [==============================] - 16s 22ms/step - loss: 0.5502 - accuracy: 0.8290 - val_loss: 0.4300 - val_accuracy: 0.8520
Epoch 3/50
748/748 [==============================] - 16s 22ms/step - loss: 0.4328 - accuracy: 0.8565 - val_loss: 0.3973 - val_accuracy: 0.8667
Epoch 4/50
748/748 [==============================] - 17s 23ms/step - loss: 0.3817 - accuracy: 0.8672 - val_loss: 0.3336 - val_accuracy: 0.8751
Epoch 5/50
748/748 [==============================] - 16s 22ms/step - loss: 0.3417 - accuracy: 0.8803 - val_loss: 0.3452 - val_accuracy: 0.8763
Epoch 6/50
748/748 [==============================] - 16s 21ms/step - loss: 0.3191 - accuracy: 0.8850 - val_loss: 0.3074 - val_accuracy: 0.8917
Epoch 7/50
748/748 [==============================] - 17s 23ms/step - loss: 0.2985 - accuracy: 0.8931 - val_loss: 0.2850 - val_accuracy:

In [22]:
for epoch, (val_loss, val_acc) in enumerate(zip(history.history['val_loss'], history.history['val_accuracy']), start=1):
    print(f"Epoch {epoch}: val_loss={val_loss:.4f}, val_accuracy={val_acc:.4f}")

score = model.evaluate(X_val, y_val, verbose=0)
print(f"Final Validation Accuracy: {score[1] * 100:.2f}%")
# 91.84

Epoch 1: val_loss=0.5065, val_accuracy=0.8375
Epoch 2: val_loss=0.4300, val_accuracy=0.8520
Epoch 3: val_loss=0.3973, val_accuracy=0.8667
Epoch 4: val_loss=0.3336, val_accuracy=0.8751
Epoch 5: val_loss=0.3452, val_accuracy=0.8763
Epoch 6: val_loss=0.3074, val_accuracy=0.8917
Epoch 7: val_loss=0.2850, val_accuracy=0.8943
Epoch 8: val_loss=0.2636, val_accuracy=0.9045
Epoch 9: val_loss=0.2796, val_accuracy=0.8926
Epoch 10: val_loss=0.2648, val_accuracy=0.8990
Epoch 11: val_loss=0.2590, val_accuracy=0.9041
Epoch 12: val_loss=0.2774, val_accuracy=0.8977
Epoch 13: val_loss=0.2505, val_accuracy=0.9084
Epoch 14: val_loss=0.2398, val_accuracy=0.9133
Epoch 15: val_loss=0.2427, val_accuracy=0.9096
Epoch 16: val_loss=0.2483, val_accuracy=0.9086
Epoch 17: val_loss=0.2779, val_accuracy=0.9017
Epoch 18: val_loss=0.2496, val_accuracy=0.9062
Epoch 19: val_loss=0.2366, val_accuracy=0.9099
Epoch 20: val_loss=0.2540, val_accuracy=0.9088
Epoch 21: val_loss=0.2280, val_accuracy=0.9141
Epoch 22: val_loss=0.2

In [10]:
test_df = df_test = image_processor.process_image_data("data/TestData", image_size)

In [11]:
X_test = test_df.iloc[:, 1:]
X_test_prepared = X_test.values.reshape(-1, *image_size, 1).astype('float32')

pred = model.predict(X_test_prepared)
predicted_classes = pred.argmax(axis=1)
predicted_labels = [df_names[i] for i in predicted_classes]

test_df['pred'] = predicted_labels

222/222 [==============================] - 1s 4ms/step


In [12]:
# model.save("models/model_9133")

INFO:tensorflow:Assets written to: models/model_9133/assets


INFO:tensorflow:Assets written to: models/model_9133/assets


In [14]:
filtered_df = test_df[test_df['pred'] == 'r']
test_df[['label', 'pred']]

,label,pred
0,Test4751.png,2
1,Test2320.png,H
2,Test4989.png,N
3,Test6146.png,s
4,Test1629.png,s
...,...,...
7095,Test2339.png,p
7096,Test4990.png,N
7097,Test5456.png,9
7098,Test4748.png,0
